# Stein's algorithm implementation

## Julia

In [ ]:
# binary GCD (aka Stein's) algorithm
# about 1.7x (2.1x) faster for random Int64s (Int128s)
function gcd(a::T, b::T) where T<:Union{Int64,UInt64,Int128,UInt128}
    a == 0 && return abs(b)
    b == 0 && return abs(a)
    za = trailing_zeros(a)
    zb = trailing_zeros(b)
    k = min(za, zb)
    u = unsigned(abs(a >> za))
    v = unsigned(abs(b >> zb))
    while u != v
        if u > v
            u, v = v, u
        end
        v -= u
        v >>= trailing_zeros(v)
    end
    r = u << k
    # T(r) would throw InexactError; we want OverflowError instead
    r > typemax(T) && throw(OverflowError())
    r % T
end

## Cython

In [1]:
%load_ext Cython

In [2]:
%%cython
from libc.math cimport sqrt

cdef int naivegcd(int a, int b):
    while b != 0:
        a, b = b, a % b
    return a

cdef int trailing_zeros(int n):
    cdef int order = 0
    while (n&1)==0:
        order += 1
        n >>= 1
    return order

cdef int gcd(int a, int b):
    if a == 0:
        return abs(b)
    if b == 0:
        return abs(a)
    cdef int za = trailing_zeros(a)
    cdef int zb = trailing_zeros(b)
    cdef unsigned int k = min(za, zb)
    cdef unsigned int u,v
    u = abs(a >> za)
    v = abs(b >> zb)

    while u != v:
        if u > v:
            u, v = v, u
        v -= u
        v >> trailing_zeros(v)

    r = u << k
    return r

cpdef double calc_pi_by_gcd(int N):
    cdef unsigned int s = 0
    for a in range(1, N+1):
        for b in range(1, N+1):
            if gcd(a, b) == 1:
                s += 1
    cdef double pi = sqrt(6.0*N**2/s)
    return pi

cpdef double calc_pi_by_naivegcd(int N):
    cdef unsigned int s = 0
    for a in range(1, N+1):
        for b in range(1, N+1):
            if naivegcd(a, b) == 1:
                s += 1
    cdef double pi = sqrt(6.0*N**2/s)
    return pi

In [3]:
%time calc_pi_by_gcd(10000)

Wall time: 12.7 s


3.141534239016629

In [4]:
%time calc_pi_by_naivegcd(10000)

Wall time: 10.6 s


3.141534239016629

## Numba

In [5]:
import math
from numba import jit
import time

@jit('i8(i8)',nopython=True)
def trailing_zeros(n):
    order = 0
    while (n & 1) == 0:
        order += 1
        n //= 2
    return order

@jit('f8(i8,i8)',nopython=True)
def gcd(a, b):
    if a == 0:
        return abs(b)
    if b == 0:
        return abs(a)
    za = trailing_zeros(a)
    zb = trailing_zeros(b)
    k = min(za, zb)

    u = abs(a >> za)
    v = abs(b >> zb)

    while u != v:
        if u > v:
            u, v = v, u
        v -= u
        v >> trailing_zeros(v)

    r = u << k
    return r


@jit('f8(i8)',nopython=True)
def calc_pi_by_gcd(N):
    s = 0
    for a in range(1, N+1):
        for b in range(1, N+1):
            if gcd(a, b) == 1:
                s += 1
    pi = math.sqrt(6*N**2/s)
    return pi

In [6]:
%time  calc_pi_by_gcd(10000)

Wall time: 26.7 s


3.141534239016629

# Numba (with Naive gcd)

In [7]:
import math
from numba import jit
import numpy as np


@jit('i8(i8,i8)', nopython=True)
def gcd(a, b):
    """
    Reference:
    https://qiita.com/wrist/items/889696a507fbc8b392e4
    http://swdrsker.hatenablog.com/entry/2017/12/20/235201
    https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.remainder.html
    """
    while b != 0:
        #a, b = b, a % b
        a, b = b, np.remainder(a, b)
    return a


@jit('f8(i8)', nopython=True)
def calc_pi_by_gcd(N):
    s = 0
    for a in range(1, N+1):
        for b in range(1, N+1):
            if gcd(a, b) == 1:
                s += 1
    pi = math.sqrt(6*N**2/s)
    return pi

In [8]:
%time calc_pi_by_gcd(10000)

Wall time: 10.1 s


3.141534239016629